# Google BigQuery

In [1]:
#source: https://www.blendo.co/blog/access-data-google-bigquery-python-r/
#https://www.kaggle.com/sohier/how-to-integrate-bigquery-pandas
import time
from google.cloud import bigquery
import pandas as pd
import math

In [4]:
from google.oauth2 import service_account

In [7]:
cd "/Users/Carla/Desktop/MSc Social Data Science/Data Analytics at Scale"

/Users/carla/Desktop/MSc Social Data Science/Data Analytics at Scale


In [8]:
#get credentials here: https://console.cloud.google.com/apis/credentials?project=github-oii-summative&folder&organizationId
credentials = service_account.Credentials.from_service_account_file('Github-Oii-Summative-983e8fb0d7b8.json')

In [23]:
project_id = 'github-oii-summative'

In [24]:
client = bigquery.Client(credentials= credentials,project=project_id)

In [53]:
# query_job = client.query("""

# SELECT *
# FROM
# `github-oii-summative.Github_Contributors.results_20181202_014906` as gh 

# """)

start = time.time()
results = query_job.to_dataframe()
done = time.time()
elapsed = done - start
print(elapsed)

475.0274031162262


In [59]:
#bigquery = pd.DataFrame(list(results))
#475 seconds = 8 minutes

In [133]:
#results = results.drop("firstnum", axis=1)
results.shape

(10211912, 4)

In [138]:
results.sort_values(by='events_actor_count', ascending=False).head(20)

,actor_login,events_actor_count,firstnum,secondnum
4127894,LombiqBot,19043010,1,19
6169363,OpenLocalizationTest,4004083,4,40
8211679,grid-bot,2096788,2,20
1063339,ssbattousai,2039017,2,20
5148234,tmtmtmtm,1578035,1,15
9232944,KenanSulayman,1573479,1,15
1063340,everypoliticianbot,1429215,1,14
41317,direwolf-github,1240492,1,12
4127895,coveralls,1112175,1,11
9232945,CodePipeline-Test,1051884,1,10


In [56]:
#results.to_csv(r'GitHub.txt', index=None, sep=',', mode='a')

# Make it run

Trying out a subset from the df

In [2]:
results = pd.read_csv("Github.txt")

In [5]:
results.sort_values(by='events_actor_count', ascending=False).head(20)

,actor_login,events_actor_count
4127894,LombiqBot,19043010
6169363,OpenLocalizationTest,4004083
8211679,grid-bot,2096788
1063339,ssbattousai,2039017
5148234,tmtmtmtm,1578035
9232944,KenanSulayman,1573479
1063340,everypoliticianbot,1429215
41317,direwolf-github,1240492
4127895,coveralls,1112175
9232945,CodePipeline-Test,1051884


In [6]:
sub_results = results[0:1000000]

In [7]:
#printing series types via numpy
sub_results["events_actor_count"].dtypes

dtype('int64')

## Pandas
Convert your number to string and use list slicing 

In [114]:
def list_slice(df):
    df["events_actor_count"] = df["events_actor_count"].astype(str)
    df["firstnum"] = df["events_actor_count"].str[0].apply(int)
    df["secondnum"] = df["events_actor_count"].str[0:2].apply(int)

## Math

List slicing has at least O(n) time complexity and the string conversion has O(n) space complexity.  Performance can be seen by the timeit function.  Here is a code for constant time and space complexity.

In [164]:
def math_o(df):
    df["firstnum_bigo"] = df["events_actor_count"].map(lambda x: x // 10 ** (int(math.log(x, 10)) - 1 + 1))
    df["secondnum_bigo"] = df["events_actor_count"].map(lambda x: x // 10 ** (int(math.log(x, 10)) - 2 + 1))

In [161]:
#bring it back to integer, since you converted to string above
results["events_actor_count"] = results["events_actor_count"].astype(int)

In [149]:
math_o(sub_results)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [201]:
sub_results.head()

,actor_login,events_actor_count,firstnum,secondnum,firstnum_bigo,secondnum_bigo
0,neimv,132,1,13,1,13.0
1,amberlea84,132,1,13,1,13.0
2,TheAdamBorek,132,1,13,1,13.0
3,BobbyTowers,132,1,13,1,13.0
4,HeathStace3004,132,1,13,1,13.0


## Cython

Adding type to increase performance
https://stackoverflow.com/questions/701322/how-can-you-get-the-first-digit-in-an-int-c

In [172]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [197]:
%timeit sub_results.apply(lambda x: cyth_dig(x['events_actor_count'], axis=1))

## Distributed computing

https://www.youtube.com/watch?v=dzYEWULDIAQ&list=PLE50-dh6JzC5zo2whIGqJ02CIhP3ysQLX&index=5

In [15]:
import pyspark as spark

In [22]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession \
    .builder \
    .appName ("test") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [30]:
df = spark.read.csv("Github.txt", header=True)

In [31]:
df.printSchema()

root
 |-- actor_login: string (nullable = true)
 |-- events_actor_count: string (nullable = true)



In [32]:
df.head(5)

[Row(actor_login='neimv', events_actor_count='132'),
 Row(actor_login='amberlea84', events_actor_count='132'),
 Row(actor_login='TheAdamBorek', events_actor_count='132'),
 Row(actor_login='BobbyTowers', events_actor_count='132'),
 Row(actor_login='HeathStace3004', events_actor_count='132')]

In [42]:
df.show()

+---------------+------------------+
|    actor_login|events_actor_count|
+---------------+------------------+
|          neimv|               132|
|     amberlea84|               132|
|   TheAdamBorek|               132|
|    BobbyTowers|               132|
| HeathStace3004|               132|
|        cloudkj|               132|
|      wangmeiqi|               132|
|       superVHS|               132|
|obiwanmikenolte|               132|
|       amdadria|               132|
|       Amrsaeed|               132|
|    AldanisVigo|               132|
|     AtoCrafter|               132|
|        POlITIC|               132|
|       nitaj96a|               132|
|           jpae|               132|
|         maater|               132|
|       miker123|               132|
|   steven-aerts|               132|
|        cummack|               132|
+---------------+------------------+
only showing top 20 rows



**Parallelization**

https://annefou.github.io/pyspark/03-pyspark_context/ <br>
https://blog.cloudera.com/blog/2015/09/how-to-prepare-your-apache-hadoop-cluster-for-pyspark-jobs/

# Make it right

Making sure that all approaches above yield the same result.

# Make it fast

https://stackoverflow.com/questions/41271299/how-can-i-get-the-first-two-digits-of-a-number

## Benchmarking and profiling

In [170]:
%load_ext memory_profiler
%load_ext line_profiler

### Pandas

In [115]:
%timeit -n 5 list_slice(results)

16.9 s ± 718 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [158]:
%timeit -n 5 list_slice(sub_results)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

1.79 s ± 65.2 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [159]:
%prun list_slice(results)

         51061622 function calls (51061607 primitive calls) in 31.781 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    8.955    4.477    8.955    4.477 {pandas._libs.lib.map_infer}
        1    6.841    6.841    6.841    6.841 {built-in method pandas._libs.lib.astype_unicode}
 10211912    5.158    0.000    7.794    0.000 strings.py:1779(f)
        2    4.087    2.044   13.192    6.596 {pandas._libs.lib.map_infer_mask}
 10212325    1.349    0.000    1.349    0.000 {built-in method builtins.isinstance}
 10211912    1.311    0.000    1.311    0.000 strings.py:1508(<lambda>)
20423946/20423931    1.287    0.000    1.287    0.000 {built-in method builtins.len}
        8    0.868    0.108    0.868    0.108 {method 'copy' of 'numpy.ndarray' objects}
        2    0.772    0.386    0.772    0.386 {pandas._libs.missing.isnaobj}
        2    0.481    0.240    0.481    0.240 function_base.py:4703(delete)
        1    0.393  

In [168]:
%memit list_slice(results)

peak memory: 1677.89 MiB, increment: 1177.16 MiB


In [169]:
%lprun -f list_slice list_slice(results)

Timer unit: 1e-06 s

Total time: 24.1751 s
File: <ipython-input-114-5a07a853c036>
Function: list_slice at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def list_slice(df):
     2         1     909786.0 909786.0      3.8      df["events_actor_count"] = df["events_actor_count"].astype(str)
     3         1   14645551.0 14645551.0     60.6      df["firstnum"] = df["events_actor_count"].str[0].apply(int)
     4         1    8619732.0 8619732.0     35.7      df["secondnum"] = df["events_actor_count"].str[0:2].apply(int)

### Math

In [153]:
%timeit -n 5 math_o(results)

17.6 s ± 632 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [157]:
%timeit -n 5 math_o(sub_results)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1.87 s ± 150 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [162]:
%prun math_o(results)

         40848674 function calls (40848670 primitive calls) in 24.867 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    8.766    4.383   23.889   11.945 {pandas._libs.lib.map_infer}
 10211912    6.162    0.000    8.011    0.000 <ipython-input-147-e176f0161497>:3(<lambda>)
 10211912    5.611    0.000    7.112    0.000 <ipython-input-147-e176f0161497>:2(<lambda>)
 20423824    3.350    0.000    3.350    0.000 {built-in method math.log}
        4    0.378    0.095    0.378    0.095 {method 'astype' of 'numpy.ndarray' objects}
        2    0.264    0.132    0.264    0.132 internals.py:375(set)
        2    0.163    0.082   24.494   12.247 series.py:2904(map)
        4    0.066    0.017    0.066    0.017 {method 'copy' of 'numpy.ndarray' objects}
        2    0.061    0.031    0.440    0.220 internals.py:577(_astype)
        2    0.021    0.010    0.352    0.176 frame.py:3105(__setitem__)
        1    0.020    0.020   

In [165]:
%memit math_o(results)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 868.30 MiB, increment: 679.06 MiB


In [167]:
%lprun -f math_o math_o(results)

Timer unit: 1e-06 s

Total time: 24.4156 s
File: <ipython-input-164-b2869c6baebf>
Function: math_o at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def math_o(df):
     2         1   12360267.0 12360267.0     50.6      df["firstnum_bigo"] = df["events_actor_count"].map(lambda x: x // 10 ** (int(math.log(x, 10)) - 1 + 1))
     3         1   12055357.0 12055357.0     49.4      df["secondnum_bigo"] = df["events_actor_count"].map(lambda x: x // 10 ** (int(math.log(x, 10)) - 2 + 1))

**Time / Memory Trade off**

# Visualizations

https://stackoverflow.com/questions/47404869/calculating-big-o-notation-with-recursion